#**Systematically Improving RAG Agents - Getting Started**

## 1. Environment Setup

First, let's make sure we have all necessary libraries set up:

In [ ]:
!pip install openai firecrawl-py chromadb

In [ ]:
import getpass
import chromadb
from chromadb.utils import embedding_functions
from firecrawl import FirecrawlApp
from typing import List
import uuid
import openai

In [ ]:
openai.api_key = getpass.getpass('Enter your OpenAI API key: ')

Enter your OpenAI API key: ··········


## 2. Fetch Content using FireCrawl

In [ ]:
# Get FireCrawl API key securely
firecrawl_api_key = getpass.getpass('Enter your Firecrawl API key')
app = FirecrawlApp(api_key=firecrawl_api_key)

# Define the URL to crawl
url = 'https://huyenchip.com/2025/01/07/agents.html'

# Scrape the content as markdown
response = app.scrape_url(url=url, params={
    'formats': ['markdown'],
})

# Extract the markdown text
text = response['markdown']

Enter your Firecrawl API key··········


## 3. Text Chunking

In [ ]:
def chunk_text(text: str, chunk_size: int, chunk_overlap: int) -> List[str]:
    chunks = []
    start = 0

    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start += chunk_size - chunk_overlap

    return chunks

# Define chunk parameters
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 200

# Split the text into chunks
chunks = chunk_text(text, CHUNK_SIZE, CHUNK_OVERLAP)

## 4. Initialize ChromaDB

In [ ]:
# Create ChromaDB client
client = chromadb.Client()

# Initialize embedding function using Sentence Transformers
embedding_function = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="all-MiniLM-L6-v2"
)

# Create a collection for this blog post
collection_name = "huyen_agents_blog"

# Delete the collection if it already exists
try:
    client.delete_collection(collection_name)
except:
    pass

# Create the collection
collection = client.create_collection(
    name=collection_name,
    embedding_function=embedding_function
)

In [ ]:
# Add each chunk to the collection
for i, chunk in enumerate(chunks):
    # Generate unique ID
    chunk_id = str(uuid.uuid4())

    # Add to collection
    collection.add(
        documents=[chunk],
        metadatas=[{"source": url, "chunk_index": i}],
        ids=[chunk_id]
    )

## 5. Query Function

In [ ]:
def query_rag(query, n_results=3):
    """
    Retrieve relevant documents from ChromaDB.

    Args:
        query (str): The query or question to search for
        n_results (int): Number of documents to retrieve

    Returns:
        dict: Contains retrieved documents
    """
    # Get relevant documents from ChromaDB
    results = collection.query(
        query_texts=[query],
        n_results=n_results
    )

    retrieved_docs = results['documents'][0]
    return {"retrieved_docs": retrieved_docs}

## 6. Test Queries

In [ ]:
# Example query
query = "What are agents in AI?"
results = query_rag(query, n_results=2)

print(f"Query: {query}\n")
print("Retrieved Documents:")
for i, doc in enumerate(results["retrieved_docs"]):
    print(f"\nDocument {i+1}:")
    print(doc)

Query: What are agents in AI?

Retrieved Documents:

Document 1:
 agent or a human operator. When comparing AI agents to human agents, keep in mind that humans and AI have very different modes of operation, so what’s considered efficient for humans might be inefficient for AI and vice versa. For example, visiting 100 web pages might be inefficient for a human agent who can only visit one page at a time but trivial for an AI agent that can visit all the web pages at once.

## Conclusion

At its core, the concept of an agent is fairly simple. An agent is defined by the environment it operates in and the set of tools it has access to. In an AI-powered agent, the AI model is the brain that leverages its tools and feedback from the environment to plan how best to accomplish a task. Access to tools makes a model vastly more capable, so the agentic pattern is inevitable.

While the concept of “agents” sounds novel, they are built upon many concepts that have been used since the early days of 

## 7. Generate Response

In [ ]:
def query_rag_with_generation(query, n_results=3, model="gpt-3.5-turbo"):
    # Get relevant documents
    results = query_rag(query, n_results)
    retrieved_docs = results["retrieved_docs"]

    # Create context from retrieved documents
    context = "\n\n".join(retrieved_docs)

    # Create prompt with context and query
    prompt = f"""Based on the following information from Chip Huyen's blog post about AI agents: {context}
             Please answer this question: {query}
             """

    # Generate response
    response = openai.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content

In [ ]:
print(query_rag_with_generation(query))

In AI, an agent is defined as anything that can perceive its environment through sensors and act upon that environment through actuators. The environment an agent operates in is specific to its use case, such as playing a game, scraping documents from the internet, or operating a self-driving car. Agents leverage AI models as their brain to plan how best to accomplish tasks, making them capable of autonomous and intelligent actions. The concept of agents integrates various foundational AI concepts, ultimately leading to the development of sophisticated, AI-powered assistants, coworkers, coaches, and more.
